In [2]:
import pyUn0 # for 20180721a-4.json
import json
import time
import datetime
import os,glob
import sys,re
import spidev
import numpy as np
import matplotlib
matplotlib.use('TkAgg') 
import math
import matplotlib.pyplot as plt
from scipy.signal import hilbert

In [ ]:
20180721a-4.json # First acqs
20191027b-1_vhdl.json # lit3rick - vhdl
verilog_minie.dat # lit3rick - verilog

In [41]:
IDLine = []
TT1 = []
TT2 = []
tmp = []
tdac = []

with open("20191027b-1_vhdl.json") as json_data:
    #DATA = {}
    d = json.load(json_data)
    json_data.close()
 

    A = d["data"]

    #print d.keys()
    nLp = int( (len(A)/2-1) ) 
    print(len(A),len(A)/2, nLp)
    for i in range( nLp ):
        if (A[2*i+1]) < 128:
        #print("first")
            value = 128*(A[2*i+0]&0b0011111) + A[2*i+1] - 512
            IDLine.append(((A[2*i+0]&0b11110000)/16  -8) /2) # Identify the # of the line
            TT1.append((A[2*i+0] & 0b00001000) / 0b1000)
            TT2.append((A[2*i+0] & 0b00010000) / 0b10000)
            tmp.append(value)
        else:
        #print("second")
            value = 128*(A[2*i+1]&0b11111) + A[2*i+2] - 512
            IDLine.append(((A[2*i+1]&0b11110000)/16 -8) /2) # Identify the # of the line
            TT1.append((A[2*i+1] & 0b00001000) / 0b1000)
            TT2.append((A[2*i+1] & 0b00010000) / 0b10000)
            tmp.append(value)
    print("Data acquired") 
    f = float(64/((1.0+int(d["registers"]["237"])))) 

    for i in range(len(IDLine)):
        if IDLine[i] < 0:
            IDLine[i] = 0
    LengthT = len(tmp)

    #self.EnvHil = self.filtered_signal
    #self.EnvHil = np.asarray(np.abs(signal.rrt(self.filtered_signal)))
 
    len_acq = LengthT
    len_line = int(LengthT)


    # Precising the DAC
    REG = [int(x) for x in d["registers"].keys() if int(x) < 100]
    REG.sort()
    dac = []
    for k in REG:
        dac.append(d["registers"][str(k)])
    # Building the DAC timeline
    tdac = []
    for pts in range(128*64): # @todo -> corriger pour avoir une ligne de 200us
        i = int(pts/5.0) # time in us

        try:
            tdac.append(4.0*d["registers"][str(i+16)])
        except:
            tdac.append(-1)

37762 18881.0 18880
Data acquired


In [43]:
plt.plot(range(128*64), tmp[0:128*64], 'b-')
plt.plot(range(128*64), tdac[0:128*64], 'g-')
plt.show()

In [44]:
toSave = np.asarray(tmp[0:128*64])
dataSet = '20191027b-1'
np.savez_compressed('20191027b-1', a=toSave, b=dataSet)

array([-266,  -10,  -12, ...,  -11,  -10,  -12])

In [47]:
def bytes_from_file(filename, chunksize=8192):
    with open(filename, "rb") as f:
        while True:
            chunk = f.read(chunksize)
            if chunk:
                for b in chunk:
                    yield b
            else:
                break
                
FN = 'verilog_minie.dat'

RES = []
i = 0

Pon = []
Poff = []
res = 0
# example:
for by in bytes_from_file(FN):
    if not(i):
        a = int(by)
        A = math.floor(a/2)
        Poff.append(3000*( A & 0b1000000) >> 6)
        Pon.append(3000*(A & 0b0100000) >> 5)
        A = A & 0b0011111

    if not(i%2):
        b = int(by)
        #B = int('{:08b}'.format(b)[::-1], 2)
        B = int(b/2)
    else:
        a = int(by)
        A = math.floor(a/2)
        Poff.append(3000*( A & 0b1000000) >> 6)
        Pon.append(3000*(A & 0b0100000) >> 5)
        A = A & 0b0011111
        #AA = int('{:07b}'.format(A)[::-1], 2)
        #AA = int.from_bytes(A, 'big')
        
        C = (A << 7) | B
        #CC = (DD << 7) | AA
        #print(i/2,": ",c, a,b,". Corrected ",A,B,". Binary:",bin(A),bin(B),bin(D),bin(C))#bin(a),bin(b))
        #print(int(i/2),hex(b),hex(a),": ","\tBinary\t:",bin(a),bin(A),"\t",bin(b),bin(B),bin(D))#bin(a),bin(b))
        #print(int(i/2),": \t\t\t",bin(A),"\t",bin(B),"\t",bin(C),hex(C),C)
        RES.append(C-2048)
        #print(res)
        res = 0
    i+=1

In [48]:
plt.plot(range(128*64), RES[0:128*64], 'b-') 
plt.show()

In [49]:
toSave = np.asarray(RES[0:128*64])
dataSet = '20200112a-1'
np.savez_compressed('20200112a-1', a=toSave, b=dataSet)